In [4]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "load_traffic_law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "precedent": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
PRECEDENT = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_precedent_documents(data_list):
    return [
        Document(
            page_content=f"{item['court']} {item['case_id']} : {item['content']}",
            metadata={
                "court": item["court"],
                "case_id": item["case_id"],
            }
        ) for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "load_traffic_law": safe_value(item.get(LAW_REFERENCES)),
            "precedent": safe_value(item.get(PRECEDENT)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "load_traffic_law"}))
    
    return documents


# 문서화 실행
term_docs = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
precedent_docs = convert_precedent_documents(load_json(file_paths["precedent"]))
car_case_docs = convert_car_case_documents(load_json(file_paths["car_case"]))
load_traffic_law_docs = convert_law_json_to_documents(load_json(file_paths["load_traffic_law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + precedent_docs + load_traffic_law_docs
all_docs = precedent_docs

In [5]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  


C:\Users\Playdata\AppData\Local\Temp\ipykernel_5772\3046172222.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [6]:
vectorstore.get()

{'ids': ['3fc7a2bb-f661-4d09-bc43-776e929ebeaa',
  'd3b6475a-a4c4-419e-9f49-149f888d878e',
  'dedab9ed-918c-473a-ba04-43ec1d7f3145',
  'aa3e7ced-5e86-4b8d-abc4-4641a662cc8a',
  '3df64a02-5841-4143-8c7d-5380e6a8ff96',
  'b60f9a39-7beb-43ad-8b60-bdc2a2c8881d',
  '3a8f3be2-0996-4b80-b4e9-45783c417182',
  'eba21538-c68c-40c6-83dd-65097af15843',
  '1e94c18f-b3f7-478f-ab93-3bed80b78edc',
  '48ee1974-22d8-4e2f-8799-9a2131028075',
  'a0370b42-9399-444b-800b-96f6a855f355',
  '808cdb8b-dedd-4e5a-8a95-4b908358c77d',
  '93f1365c-51ce-43e3-a891-dec8e2d5e7c1',
  '31c71bcd-5599-4b2e-a60b-723d79d92243',
  'c5dafb2b-7c13-49a9-a2f5-e04fec60dc64',
  '07d1ea6f-4834-4b78-abcb-ed5015cbc36d',
  '26f31fd5-356f-4422-bb8d-887f1d6037b7',
  '473ab361-efeb-4b82-a33e-3415f0e2400a',
  '2f5fae7f-6337-476c-ad91-63fdb7ad2789',
  'df0bdcaf-e8f2-4f7a-b086-fa5f59407f73',
  '997e8fb1-ea12-48bf-afe8-8cee6fb09947',
  'eb539d73-78e1-4bfa-891a-395b35eb968f',
  '2d34a2ee-2990-4ef0-8ddf-b903bdffe766',
  'b2c5043e-78ee-48df-98cf-

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import re

# LLM 모델 2개 설정
expert_model = ChatOpenAI(model="gpt-4o", temperature=0.5)
casual_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# 분류용 모델도 전문가 모델을 활용
classification_model = expert_model

# 사고 분류 (accident인지 detail인지 구분)
classification_prompt = PromptTemplate.from_template("""
너는 교통사고 상담 챗봇의 질문 분류기야.

사용자의 질문이 다음 중 어떤 유형인지 판단해:

1. 사고 상황 판단 질문 (예: 사고 경위 설명, 과실비율 요청)
2. 법률/판례/용어 설명 질문 (예: 도로교통법 조항, 판례 번호, '과실' 용어 정의 등)
3. 일반 질문 (예: "너는 누구야?", "날씨 어때?", "GPT란 뭐야?" 등 교통사고와 무관한 질문)

출력은 반드시 아래 중 하나만 해:
- accident
- detail
- general

다른 말 없이 위 세 단어 중 하나만 정확히 출력해.

질문:
{question}

출력:
""")

# 일반 질문에 대한 프롬프트
general_prompt = PromptTemplate.from_template("""
너는 교통사고 상담 전문 AI 챗봇이야.

교통사고 판례, 도로교통법, 법률 용어 등에 대해 사용자에게 도움을 주는 역할을 해.

하지만 아래와 같은 상황에서도 혼자 유연하게 답변해야 해:
- 문서에서 찾을 수 없는 용어, 법률 조항, 판례 번호가 나왔을 경우
- 문서에 없는 질문이라도, 너의 지식과 상식으로 설명이 가능한 경우
- 문서와 상관없는 일반적인 질문 (예: 자기소개, 인공지능, 날씨 등)

이럴 땐 “문서에 포함되어 있지 않습니다” 같은 말은 하지 말고,
AI 챗봇으로서 너의 지식으로 최대한 정확하고 자연스럽게 답변해줘.

질문:
{question}

답변:
""")



# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
아래 '사고 상황'을 분석하여 핵심 요소를 구조화하고, 반드시 문서 내에서 가장 유사한 사례(case)를 찾아 과실비율을 판단해줘.

---
사고 상황 원문:
{question}

➤ 사고 상황 요약 (다음 항목 기준):
- A차량 신호 및 진행 방식:
- B차량 신호 및 진행 방식:
- 충돌 방식 및 위치:
- 교차로/신호기 유무 등 도로 환경:

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률 (다음 형식으로 기재)
   - [법률명] 제[조]조 [항] (예: 도로교통법 제5조 제1항)
   - 여러 개면 각각 별도 줄로 나열
4. 참고 판례 (다음 형식으로 기재)
   - [법원명] [사건번호] (예: 대법원 2002다38767)
   - 여러 개면 각각 별도 줄로 나열

조건:
- 반드시 문서 내 유사 사례를 기반으로 판단해야 해.
- 사고 상황과 가장 유사한 사례를 먼저 정확히 찾고, 그 사례를 중심으로 판단 근거를 제시해.
- 직접적인 사례가 없으면, 가장 유사한 사례를 근거로 판단하고 그 차이를 설명해.
- 과실비율은 명확한 수치로 제시해.
- 법령, 판례는 반드시 문서에서 인용된 것을 정확한 형식으로 명시해.
- 무조건적인 추론이나 일반적인 상식은 사용하지 말고, 문서에 있는 정보로만 판단해.
- 판례명을 포함한 질문인 경우, 문서에서 사건번호와 법원명이 일치하는 문장을 찾아 정확히 인용해줘.
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
# detail_prompt = PromptTemplate(
#     template="""
# 너는 교통사고와 법률 해설 전문 챗봇이야.

# 아래 문서 내용을 바탕으로 사용자 질문에 정확하고 간결하게 답변해줘.

# 질문:
# {question}

# --- 문서 내용 ---
# {context}
# ------------------

# 📌 답변 작성 규칙:
# 1. 질문에 판례명(법원명 + 사건번호)이 포함된 경우:
#    - 반드시 위 문서 내에서 **해당 사건번호가 포함된 문장**을 직접 인용해 설명해.
#    - 문서에 해당 판례가 없으면 "**해당 판례는 문서에 포함되어 있지 않습니다**"라고 말해.

# 2. 질문이 용어나 법률 조항에 대한 경우:
#    - 반드시 위 문서에 포함된 문장을 바탕으로 정의하거나 설명해.
#    - 법률명, 조문 번호, 관련 판례명을 반드시 함께 명시해.

# 3. 문서에 없는 정보는 절대로 추측하거나 생성하지 마.

# ✍️ 출력 형식:
# - 정의 또는 설명: [내용]
# - 출처: [법률/조문 번호/판례명]

# 답변:
# """,
#     input_variables=["question", "context"]
# )

detail_prompt = PromptTemplate(
    template="""
너는 교통사고와 법률 해설 전문 챗봇이야.

아래 문서 내용은 교통사고 관련 판례, 법률 조항, 용어 정의가 포함된 데이터야.
문서는 다음과 같은 형식의 메타데이터를 포함하고 있어:

📘 판례 메타데이터 예시:
- case_id: '2006나4320'
- court: '서울고등법원'
- type: 'judgment'

📘 법률 조항 예시:
- 도로교통법 제5조 제1항
- 도로교통법 제25조 제5항

📘 용어 정의는 '과실', '중대한 과실', '현저한 과실' 등 문장으로 설명됨

사용자의 질문을 보고 다음 중 하나에 해당하도록 답변해줘:
1. 판례(case_id)가 질문에 포함되면, 문서에서 해당 case_id가 포함된 문장을 찾아서 **직접 인용**하여 설명해.
2. 법률 조항(예: 도로교통법 제x조 제y항)이 포함된 경우, 반드시 **문서에 있는 원문**을 바탕으로 설명하고 법률명을 정확히 명시해.
3. 용어나 개념 정의 질문일 경우, 문서에서 해당 용어가 포함된 문장을 찾아 그 문장만 바탕으로 설명해.

⚠️ 문서에 포함되지 않은 정보는 생성하지 마.
⚠️ 질문과 관련된 메타데이터가 없으면, “문서에 포함되어 있지 않습니다”라고 명시해.

✍️ 출력 형식:
- 정의 또는 설명: [내용]
- 출처: [법률/조문 번호/판례명]

질문:
{question}

--- 문서 내용 ---
{context}
------------------

답변:
""",
    input_variables=["question", "context"]
)


# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=expert_model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=expert_model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# 통합 실행 함수
# 🔍 판례명 추출
def extract_case_id(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)\s?\d{4}[가-힣]?\d+", text)
    return match.group() if match else None

# 🔍 법원명 추출
def extract_court_name(text: str):
    match = re.search(r"(대법원|[\w]+고등법원|[\w]+지방법원)", text)
    return match.group() if match else None

# 질문에 대한 accident인지 detail인지 분류 함수
def classify_query_llm(user_input: str):
    prompt = classification_prompt.format(question=user_input)
    result = classification_model.invoke(prompt)
    return result.content.strip().lower()

# 일반적인 질문에 대한 함수
def handle_general_question(user_input: str):
    prompt = general_prompt.format(question=user_input)
    result = casual_model.invoke(prompt)
    print(result.content.strip())


# ✨ context 최적화 함수
def build_context_from_matched_docs(matched_docs, key_phrase=None, limit=3):
    if key_phrase:
        primary_docs = [doc for doc in matched_docs if key_phrase in doc.page_content]
        other_docs = [doc for doc in matched_docs if key_phrase not in doc.page_content]
        ordered_docs = primary_docs + other_docs[:limit - len(primary_docs)]
    else:
        ordered_docs = matched_docs[:limit]

    # ✅ 각 문서 최대 800자 제한
    context = "\n\n".join(doc.page_content[:800] for doc in ordered_docs)
    return context


def run_query(user_input: str):
    case_id = extract_case_id(user_input)
    court = extract_court_name(user_input)

    # ① 판례 검색
    if case_id:
        matched_docs = [doc for doc in all_docs if case_id in doc.page_content]
        if matched_docs:
            context = build_context_from_matched_docs(matched_docs, key_phrase=case_id)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = expert_model.invoke(prompt_text)
            print(result.content.strip())
            return
        else:
            # 문서 없음 → 일반 응답
            handle_general_question(user_input)
            return

    # ② 법원명 검색
    if court:
        matched_docs = [doc for doc in all_docs if doc.metadata.get("court") == court]
        if matched_docs:
            context = build_context_from_matched_docs(matched_docs)
            prompt_text = detail_prompt.format(question=user_input, context=context)
            result = expert_model.invoke(prompt_text)
            print(result.content.strip())
            return
        else:
            # 문서 없음 → 일반 응답
            handle_general_question(user_input)
            return

    # ③ LLM 분류
    try:
        category = classify_query_llm(user_input).strip().lower()
    except:
        category = None

    if category == "accident":
        result = accident_chain.run(user_input)
        print(result)
        return

    elif category == "detail":
        matched_docs = retriever.get_relevant_documents(user_input)

        # detail인데 문서 없음 → 일반 응답으로 fallback
        if not matched_docs or all(doc.page_content.strip() == "" for doc in matched_docs):
            handle_general_question(user_input)
            return

        # ✅ 문서 있을 때만 detail_chain 실행
        result = detail_chain.run(user_input)
        print(result)
        return

    # accident/detail 외 → 일반 응답
    handle_general_question(user_input)


### 테스트

In [62]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

1. 과실비율: A차량 100% vs B차량 0%

2. 판단 근거 요약:
   - A차량은 적색신호에서 직진하였으므로 신호 위반으로 인한 과실이 큽니다.
   - B차량은 녹색신호에서 좌회전하였으므로 신호에 따른 정상적인 진행으로 간주됩니다.
   - 문서 내 유사 사례인 인천지방법원 93가단5580 사건에서는 A차량이 신호를 위반하여 직진 중 사고를 유발하였고, B차량은 녹색신호에 따른 정상적인 좌회전 중이었으므로 B차량의 과실이 0%로 판단되었습니다.

3. 적용 법률:
   - [도로교통법] 제5조 제1항 (신호에 따른 통행의무)

4. 참고 판례:
   - 인천지방법원 93가단5580


In [63]:
run_query("비보호 좌회전이 뭔지 설명해줘")

- 정의 또는 설명: 비보호 좌회전은 신호등이 있는 교차로에서 좌회전 신호가 아닌 경우에도 좌회전을 할 수 있도록 허용된 상황을 말합니다. 비보호 좌회전을 하는 차량은 반대방향에서 직진 신호에 따라 진행하는 차량에 방해가 되지 않도록 주의하면서 조심스럽게 좌회전해야 할 의무가 있습니다.
- 출처: [서울중앙지방법원 2017나65373, 대구지방법원 2016나309440, 서울남부지방법원 2015가단227332, 광주지방법원 2015가단29829]


In [36]:
run_query("대법원 92도2077 판결문 내용을 알려줘")

- 정의 또는 설명: 차량의 운전자로서는 횡단보도의 신호가 적색인 상태에서 반대 차로 상에 정지하여 있는 차량의 뒤로 보행자가 건너오지 않을 것이라고 신뢰하는 것이 당연하고 그렇지 아니할 사태까지 예상하여 그에 대한 주의의무를 다하여야 한다고는 할 수 없다.
- 출처: 대법원 92도2077


In [37]:
run_query("서울남부지방법원에는 어떤 판례가 있어?")

- 정의 또는 설명: [서울남부지방법원에는 다음과 같은 판례가 있습니다: 
  1. 2018나64069: 신호등이 있는 ‘ㅏ’자형 교차로에서 전방 신호기 황색등화에 교차로를 가속 진입한 A차량과 반대편 차선 좌회전 금지구역에서 적색 신호에 좌회전을 한 B이륜차 간의 충돌로 B이륜차 운전자가 사망한 사고: B과실 55%(안전모 미착용 포함).
  2. 2015가단227332: 주간에 신호등 및 비보호 좌회전 표지가 있는 삼거리(T자) 교차로에서 A차량이 선행 차량을 따라 서행으로 비보호 좌회전 하던 중 맞은편에서 직진신호에 따라 주행하여 오는 차량의 흐름을 잘 살펴 그 운행에 방해되지 않는 방법으로 안전하게 좌회전할 의무를 게을리 한 과실로, 교차로에 진입하기 전에 먼저 비보호 좌회전 하는 차량(제3차량)이 있었음에도 전혀 속도를 줄이지 아니한 채 만연히 신호에 따라 빠른 속도로 교차로에 진입하여 직진하던 B차량을 충격한 사고: B과실 10%.
  3. 2018나11137: B 직진차량이 규정 속도 이내이고 양 차량 동시 교차로 진입하였고 A 비보호 좌회전 차량이 선행하던 직진 차량 뒤에서 급진입하여 좌회전한 경우 B차량이 A차량의 비보호 좌회전과의 충돌을 인지하고 브레이크를 밟아 제동이 시작되기까지의 공주거리에도 못 미치는 약 18m 전방에서 발견한 경우로, B차량 0%.(1심은 B차량 10%)]
- 출처: [서울남부지방법원 판례]


In [53]:
run_query("A차가 직진 중이고 B차가 좌회전하다 충돌했을 때 과실은?")

사고 상황을 분석하여 과실비율을 판단하겠습니다.

➤ 사고 상황 요약:
- A차량 신호 및 진행 방식: 직진 중
- B차량 신호 및 진행 방식: 좌회전 중
- 충돌 방식 및 위치: A차와 B차가 충돌
- 교차로/신호기 유무 등 도로 환경: 정보 없음

문서에서 가장 유사한 사례를 찾겠습니다.

문서 내에서 가장 유사한 사례는 "서울중앙지방법원 2015나60480"입니다. 이 사례에서는 신호등이 있는 교차로에서 직진전용차로를 진행하던 B차량이 좌회전전용차로에서 같은 방향으로 좌회전하고 있던 A차량을 추월하여 발생한 사고에서 B과실이 80%로 판결되었습니다.

1. 과실비율: A차량 20% vs B차량 80%

2. 판단 근거 요약:
   - B차량이 좌회전 중 A차량과 충돌한 상황에서, B차량의 진행 방향과 신호 준수 여부가 과실 판단에 중요한 요소로 작용합니다.
   - 유사 사례에서 B차량의 과실이 80%로 인정된 점을 고려하여, A차량 20%, B차량 80%로 과실비율을 판단합니다.

3. 적용 법률:
   - 도로교통법 제5조 제1항 (신호 준수 의무)
   - 도로교통법 제25조 제1항 (진로 변경 시 주의 의무)

4. 참고 판례:
   - 서울중앙지방법원 2015나60480

위 사례와 사고 상황의 유사성을 바탕으로 과실비율을 판단하였습니다.


In [54]:
run_query("서울남부지방법원 2018나64069 판결문 내용을 알려줘")

- 정의 또는 설명: 신호등이 있는 ‘ㅏ’자형 교차로에서 전방 신호기 황색등화에 교차로를 가속 진입한 A차량과 반대편 차선 좌회전 금지구역에서 적색 신호에 좌회전을 한 B이륜차 간의 충돌로 B이륜차 운전자가 사망한 사고: B과실 55%(안전모 미착용 포함)
- 출처: 서울남부지방법원 2018나64069


In [66]:
run_query("너는 누구야?")

저는 교통사고 상담 전문 AI 챗봇이에요. 교통사고 판례, 도로교통법, 법률 용어 등에 대해 도움을 드리고 있어요. 무엇을 도와드릴까요?


In [67]:
run_query("날씨가 어때?")

지금은 교통사고에 관련된 내용에 대해 도와드릴 수 있는 상황이에요. 날씨 정보는 인터넷 기상 서비스를 통해 확인해 주시기 바랍니다. 다른 궁금한 점이 있으시면 언제든지 물어봐주세요!


In [31]:
import random
json_precedent = load_json(file_paths["precedent"])
random_precedent = random.sample(json_precedent, 10)

for precedent in random_precedent:
    run_query(f"\n{precedent['court']} {precedent['case_id']} 판결문 내용을 알려줘")
    print()

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 주간에 신호등 있는 사거리(十자) 교차로에서 B차량이 좌회전 하던 중 전방주시 및 보행자 보호의무를 소홀히 한 과실로 때마침 횡단보도의 녹색신호에 진입하여 신호가 적색으로 바뀔 때까지 미처 도로를 횡단하지 못한 A(노인)를 사고차량의 좌측면으로 충격한 후 넘어진 A의 우측손을 사고차량의 뒷바퀴로 역과하여 상해를 입게하여 치료 도중 사망에 이르게 한 사고 : A과실 20%
- 출처: [서울고등법원 2002나4535]

📘 판례 직접 검색 결과:
 문서에 포함되어 있지 않습니다.

📘 '서울중앙지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 문서에 포함되어 있지 않습니다.
- 출처: 서울중앙지방법원 2007가단290391 판결문

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 피고 차량이 적색점멸신호가 작동하는 도로에서 일시정지하지 않은 채 제한속도를 시속 20㎞ 이상 위반한 과속 상태로 교차로에 진입하면서 황색점멸신호가 작동하는 도로에서 진입한 원고 차량을 충격한 사고에 대하여 피고 차량이 일시정지 없이 과속 상태로 주행한 과실 인정하되 다만 원고 차량도 황색점멸신호에 진행하면서 피고 차량의 진행상황에 대한 주의를 소홀히 한 잘못이 인정된다고 보아 피고 차량의 책임을 일부 제한함. 원고 차량 과실 10%, 피고 차량 과실 90%
- 출처: [대전지방법원 2014나9524]

📘 판례 직접 검색 결과:
 - 정의 또는 설명: 주간에, A차량이 매장 앞에서 물건을 하차하기 위해 삼거리(T자) 교차로의 모서리 부근에 있는 횡단보도를 걸쳐 주·정차 중이었는데, B차량이 후진하면서 A차량을 충격한 사고에서, 도로교통법 제32조에 따라 삼거리(T자) 교차로의 모서리 부근에 있는 횡단보도를 걸친 불법 주·정차 과실 역시 사고 발생 및 손해 확대의 원인이 되었으므로 A 과실 20%.
- 출처: 서울중앙지방법원 2018나62265

📘 '서울동부지방법원' 소속 판례 검색 결과:
 - 정의 또는 설명: 주간에 도로 외

In [ ]:
!pip install lark